# Selección Bayesiana de modelos


## Autores

1. Alvaro Mauricio Montenegro Díaz, ammontenegrod@unal.edu.co

# Referencias

1. Alvaro Montenegro, [Curso de Estadística Bayesiana](https://github.com/AprendizajeProfundo/Estadistica-Bayesiana), 2020
2. Guangyuan Gao, Bayesian Claims Reserving Methods in Non-life Insurance with Stan. An Introduction, Springer, 2018
3. ["Stan User's Guide. Version 2.22", Stan Development Team, 2020.](https://mc-stan.org/users/documentation/)

# Introducción

En esta lección revisamos herramientas estadísticas basadas en criterios de información para comparar modelos. 

En ocasiones es necesario seleccionar un modelo entre varios plausibles. Cuando se tiene varios modelos factibles para los datos y no se requiere seleccionar uno en particular, es mejor utlizarlos todos, mediante la técnica de promedio de modelos que veremos en otra lección.

# Selección de modelos

In [3]:
stacks_lasso = '''
data {
  int<lower=0> N;
  int<lower=0> p;
  real Y[N];
  matrix[N,p] x;
} 

// to standardize the x's 
transformed data {
  real z[N,p];
  real mean_x[p];
  real sd_x[p];
  for (j in 1:p) { 
    mean_x[j] <- mean(col(x,j)); 
    sd_x[j] <- sd(col(x,j)); 
    for (i in 1:N)  
      z[i,j] <- (x[i,j] - mean_x[j]) / sd_x[j]; 
  }
}

parameters {
  real beta0; 
  real beta[p]; 
  real<lower=0> sigmasq; 
  real<lower=0> phi;
} 

transformed parameters {
  real<lower=0> sigma;
  real mu[N];

  sigma <- sqrt(2) * sigmasq;
  for (n in 1:N)
    mu[n] <- beta0 + beta[1] * z[n, 1] + beta[2] * z[n, 2] + beta[3] * z[n, 3];
}

model {
  beta0 ~ normal(0, 316); 
  phi ~ gamma(0.01, 0.01);
  beta ~ normal(0, sqrt(phi));
  sigmasq ~ inv_gamma(.001, .001); 
  for (n in 1:N) 
    Y[n] ~ double_exponential(mu[n], sigmasq); 
} 

generated quantities {
  real b0;
  real b[p];
  real outlier_1;
  real outlier_3;
  real outlier_4;
  real outlier_21;

  for (j in 1:p)
    b[j] <- beta[j] / sd_x[j];
  b0 <- beta0 - b[1] * mean_x[1] - b[2] * mean_x[2] - b[3] * mean_x[3];

  outlier_1  <- step(fabs((Y[1] - mu[1]) / sigma) - 2.5);
  outlier_3  <- step(fabs((Y[3] - mu[3]) / sigma) - 2.5);
  outlier_4  <- step(fabs((Y[4] - mu[4]) / sigma) - 2.5);
  outlier_21 <- step(fabs((Y[21] - mu[21]) / sigma) - 2.5);
}
'''

El problema de selección del modelo es una negociación (trade-off) entre un modelo simple y un buen ajuste.

Idealmente, queremos elegir el modelo más simple con el mejor ajuste.

Sin embargo, los modelos que se adaptan bien tienden a ser más complicados, mientras que los modelos más simples tienden a ser poco adecuados.

Los métodos de selección de modelos utilizados en la estadística frecuentista suelen ser los de de validación cruzada y los basados en criterios de información, que son la suma residual de cuadrados modificada con respecto a la complejidad del modelo y al sobreajuste.

La validación cruzada mide el ajuste de un modelo en el conjunto de datos de prueba, que no es usado para ajustar el modelo, mientras que los criterios de información ajustan la medida de ajuste en el conjunto de datos de entrenamiento agregando una penalización por la complejidad del modelo.


# La precisión predictiva de un modelo

En el marco Bayesiano, el ajuste de un modelo a veces se llama 
precisión predictiva del modelo (Gelman et al. 2014). Medimos la precisión predictiva de un modelo para un conjunto de datos $\mathbf{z}$ por el logaritmo de densidad predictiva puntual (log point wise predictive density) o **lppd**,  el cual es calculado como sigue. Supongamos que $\mathbf{y}$ es el conjunto de datos observados. Idealmente $\mathbf{z}$ no debería ser usudo para ajustar el modelo.


$$
\text{lppd}:= \log \prod_{i=1}^{m} \mathbb{E}_{\boldsymbol{\theta}| \mathbf{y}} p( \mathbf{z}|\boldsymbol{\theta}) = \sum_{i=1}^{m} \log (\mathbb{E}_{\boldsymbol{\theta}| \mathbf{y}} p(\mathbf{z}|\boldsymbol{\theta})) = \sum_{i=1}^{m} \log \left( \int p(\mathbf{z}|\boldsymbol{\theta}) p(\theta| \mathbf{y})d\boldsymbol{\theta} \right).
$$

Si escogemos  $\mathbf{z} = \mathbf{y}$, obtenemos el lppd dentro de la muestra (within-sample) y los denotamos por $lppd_{train}$, el cual es típicamente mayor que el lppd fuera de muestra (out-of-sample)  el cual dnotamos por $lppd_{test}$. 



Para calcular **lppd** en la práctica, podemos evaluar la esperanza utilizando muestras de la distribución posterior $ p(\boldsymbol{\theta} | \mathbf{y}) $, que etiquetamos como $ \boldsymbol{\theta}^{(t)}, t = 1, \ldots, T$. El lppd calculado se define de la siguiente manera:


$$
\text{computed } lppd:= \sum_{i=1}^{n} \log\left( \frac{1}{T} \sum_{t=1}^{T} p(\mathbf{z}|\boldsymbol{\theta}^{(t)})\right).
$$

## Validación cruzada

En la validación cruzada Bayesiana, los datos se dividen repetidamente en un conjunto de entrenamiento $\mathbf{y}_{train} $ y un conjunto de pruebas $\mathbf{y}_{test}$.


Para simplificar, restringimos nuestra atención a la validación cruzada de dejar uno fuera, leave one out cross-validation(LOO-CV), donde $\mathbf{y}_{test}$ solo contiene un punto de los datos. La estimación Bayesian LOO-CV de lppd fuera de muestra se define de la siguiente manera:

$$
lppd_{loo-cv} := \sum_{i=1}^{n} \log \left( \int p (y_i |\boldsymbol{\theta}) p (\boldsymbol{\theta}|\mathbf{y}_{−i} ) d\boldsymbol{\theta}\right).
$$

en donde $\mathbf{y}_{−i}$ es el conjunto de datos omitiendo el  $i$-th punto. The $lppd_{loo-cv}$ puede calcularse como

$$
\text{computed } lppd_{loo-cv} = \sum_{i=1}^{n} \log  \left(\frac{1}{T} \sum_{t=1}^{T} p(y_i|\theta^{(it)}) \right),
$$

en donde $\theta^{(it)}, t = 1, \ldots , T$ son simulaciones de la distribución posterior $p(\theta|\mathbf{y}_{-i})$.

# Criterios de información para selección de modelos

Antes de describir el DIC, revisamos otros dos criterios de información empleados en
estadística frecuentistas. El criterio de información de Akaike (AIC) de Akaike (1973) es
definido como

$$
AIC:= − 2\sum_{i=1}^{n} \log p (y_i |\boldsymbol{\theta}_{MLE}) + 2p.
$$



El criterio de información Bayesiano (BIC) de Schwarz (1978) se define como


$$
BIC:= − 2\sum_{i=1}^{n} \log p (y_i |\boldsymbol{\theta}_{MLE}) + p\log n,
$$

en donde $ p $ es el número de parámetros. El primer término común $ - 2 \sum_ {i = 1}^ {n} \log p (y_i | \boldsymbol{\theta}_ {MLE}) $ es la deviance, la cual  mide la discrepancia entre el modelo ajustado y los datos. El segundo término mide la complejidad del modelo.

## DIC: Deviance Information Criterion DIC


En el marco Bayesiano, definimos una cantidad similar para medir la discrepancia,
$ -2 \sum_ {i = 1} ^ {n} \log p (y_i | \hat{\theta}) $, donde $ \hat{\theta} $ es la media posterior. 

Spiegelhalter et. al. (2002) propusieron una medida del número de parámetros efectivos, que se define como la diferencia entre la media posterior de la desviación y la desviación en la media posterior, como sigue:


$$
p_D := \widehat{D (\boldsymbol{\theta})} - D(\hat{\boldsymbol{\theta}}) = −2\mathbb{E}_{\theta|y} \left(\sum_{i=1}^{n} \log p (y_i |\boldsymbol{\theta})\right) + 2 \sum_{i=1}^{n} \log p(y_i|\hat{\boldsymbol{\theta}}),
$$

en donde $D$ es la deviance definida arriba.

Observe que el cálculo práctico se tiene que si $\boldsymbol{\theta}^{(t)}$ es una muestra de tamaño $T$ de la posterior $p(\boldsymbol{\theta}|\mathbf{y})$, entonces


$$
\begin{align}
\widehat{D (\boldsymbol{\theta})}&\approx \frac{-2}{T}\sum_{t=1}^{T}\left(\sum_{i=1}^{n} \log p (y_i |\boldsymbol{\theta}^{(t)})\right)\\
\hat{\boldsymbol{\theta}}&\approx \frac{1}{T}\sum_{t=1}^{T} \boldsymbol{\theta}^{(t)}.
 \end{align}
$$

Los autores propusieron además un criterio de información de deviance (DIC), definido como
la deviance en las medias posteriores más el doble del número efectivo de parámetros,
con lo que se obtiene

$$
DIC := D( \hat{\boldsymbol{\theta}}) + 2p_D.
$$


DIC es visto como un análogo Bayesiano de AIC. Preferimos el modelo con menor DIC.

Hay que tener en cuenta que DIC y $ p_D $ son sensibles al nivel en un modelo jerárquico. 

Entonces DIC es más apropiado cuando estamos interesados en los parámetros directamente relacionados con los datos.

## WAIC: Criterio de información de  Watanabe-Akaike (widely available information criterion)

Watanabe (2010) propuso otra medida del número de parámetros efectivos de la siguiente manera:

$$
p_{WAIC} := \widehat{D(\theta)} + 2lppd_{train} = −2\mathbb{E}_{\theta|y} \left(\sum_{i=1}^{n} \log p (y_i |\theta) \right) + 2 \sum_{i=1} ^{n} \log \left( \mathbb{E}_{\theta|y} p (y_i |\theta)\right),
$$

en donde $−2lppd_{train}$ juega un rol como $D(\hat{\theta})$ en  $p_D$. Como con AIC y DIC, WAIC se define por

$$
WAIC:= − 2lppd_{train} + 2p_{WAIC}.
$$

Se ha encontrado que este criterio es en general más robusto que el DIC.

## Criterio Leave-One-Out Information Criterion (LOOIC)


Diferente de la definición del número de parámetros efectivos en AIC, DIC y
WAIC, definimos

$$
p_{loo} := lppd_{train} − lppd_{loo-cv},
$$

en donde $lppd_{loo-cv}$ se calcula como se muestra arriba. The criterio leave-one-out information criterion (LOOIC) se define como

$$
LOOIC:= − 2lppd_{train} + 2p_{loo} = −2lppd_{loo-cv},
$$)
el cual es razonable debido a que $lppd_{loo-cv}$ ya penaliza el sobreajuste (overfitting)
(o de manera equivalente la complejidad del modelo).

# Tarea

Para los modelos propuestos para los datos de stack loss, escriba el código para calcular los criterios DIC, WAIC y LOOIC.

Haga una tabla comparativa que colocando acad modelo, el número de parámetros efectivos y el valor del criterio.